# Preparation

In [1]:
!pip install -qU sentence-transformers
!pip install -qU wikipedia-api
!pip install -qU hazm
!pip install -qU clean-text[gpl]
!pip install -qU emoji

     |████████████████████████████████| 71kB 3.5MB/s 
     |████████████████████████████████| 1.3MB 19.2MB/s 
     |████████████████████████████████| 1.1MB 48.4MB/s 
     |████████████████████████████████| 2.9MB 47.2MB/s 
     |████████████████████████████████| 890kB 43.9MB/s 
     |████████████████████████████████| 317kB 5.7MB/s 
     |████████████████████████████████| 1.4MB 10.1MB/s 
     |████████████████████████████████| 235kB 19.5MB/s 
     |████████████████████████████████| 51kB 1.9MB/s 
     |████████████████████████████████| 71kB 3.9MB/s 
     |████████████████████████████████| 245kB 6.5MB/s 


In [2]:
!mkdir resources
!wget -q "https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip" -P resources
!unzip -qq resources/resources-0.5.zip -d resources

In [3]:
!rm -rf /content/4ccae468eb73bf6c4f4de3075ddb5336
!rm -rf /content/preproc
!rm preprocessing.py utils.py
!mkdir -p /content/preproc
!git clone https://gist.github.com/4ccae468eb73bf6c4f4de3075ddb5336.git /content/preproc/
!mv /content/preproc/* /content/
!rm -rf /content/preproc

rm: cannot remove 'preprocessing.py': No such file or directory
rm: cannot remove 'utils.py': No such file or directory
Cloning into '/content/preproc'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (7/7), done.


In [4]:
!wget -q https://raw.githubusercontent.com/UKPLab/sentence-transformers/master/examples/applications/text-summarization/LexRank.py

In [7]:
from preprocessing import cleaning
from IPython import display

import nltk
import wikipediaapi

import numpy as np
import pandas as pd

import hazm
import requests
import time

import torch
from sentence_transformers import models, SentenceTransformer, util
from LexRank import degree_centrality_scores

In [8]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [9]:
wiki = wikipediaapi.Wikipedia('fa')

In [10]:
def rtl_print(outputs, font_size="15px", n_to_br=False):
    outputs = outputs if isinstance(outputs, list) else [outputs] 
    if n_to_br:
        outputs = [output.replace('\n', '<br/>') for output in outputs]
        
    outputs = [f'<p style="text-align: right; direction: rtl; margin-right: 10px; font-size: {font_size};">{output}</p>' for output in outputs]
    display.display(display.HTML(' '.join(outputs)))


FILTERD_SECTIONS = [
    # 'تاریخچه',
    'محتویات',
    'پانویس',
    'منابع',
    'منابع و پانویس',
    'جستارهای وابسته',
    'پیوند به بیرون',
    'یادداشت‌ها',
    'یادداشت‌ ها',
    'جوایز',
    'نگارخانه',
    'روابط‌خارجی',
    'روابط خارجی',
    'کتاب‌شناسی',
    'کتاب‌ شناسی',
    'فیلم‌شناسی',
    'فیلم ‌شناسی',
    'دست‌اندرکاران',
    'دست‌اندر کاران',
    'دست‌ اندر کاران',
    'فروش‌های برگزیدهٔ آلبوم',
    'فروش‌ های برگزیدهٔ آلبوم',
    'فروش‌ های برگزیده آلبوم',
    'نمودارهای فروش',
    'نمودار های فروش',
    'فهرست آهنگ‌ها',
    'فهرست آهنگ‌ ها',
    'اعضا',
    'ترانه‌شناسی',
    'ترانه‌ شناسی',
    'نگارخانه',
    'بازیگران',
    'پروژه‌های مشابه',
    'پروژه‌ های مشابه'
]
FILTERD_SECTIONS = [t.strip() for t in list(set(FILTERD_SECTIONS))]

def get_sections(sections, level=0, retrieved=None, keep_title=False):
    retrieved = retrieved if isinstance(retrieved, list) else []

    for s in sections:
        if s.title not in FILTERD_SECTIONS:
            if keep_title:
                text = s.title + '\n' + s.text
            else:
                text = s.text
            
            retrieved.append(text.replace('\n', '[n]'))
            get_sections(s.sections, level=level + 1, retrieved=retrieved, keep_title=keep_title)
    
    return '[nn]'.join(retrieved)

def wiki_document(page_name, make_clean=True, verbose=False, keep_title=True, keep_new_lines=True, n1_rp='\n', n2_rp='\n\n'):
    page_py = wiki.page(page_name)
    document = ''

    if page_py.exists():
        document = []
        document = get_sections(page_py.sections, level=0, retrieved=document, keep_title=keep_title)
        document = cleaning(document) if make_clean else document
        document = document.replace('[nn]', n2_rp).replace('[n]', n1_rp) if keep_new_lines else docuemnt 

        if verbose:
            rtl_print("Page - Title: %s" % page_py.title)
            rtl_print("%s" % requests.utils.unquote(page_py.fullurl))
            rtl_print("Page - Article [1000]: %s ..." % cleaning(document[:1000]))
    
    return document


def load_st_model(model_name_or_path):
    word_embedding_model = models.Transformer(model_name_or_path)
    pooling_model = models.Pooling(
        word_embedding_model.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True,
        pooling_mode_cls_token=False,
        pooling_mode_max_tokens=False)
    
    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    return model

# Text Summarization

In [11]:
document = wiki_document('هوش_مصنوعی', make_clean=True, verbose=True, keep_title=False, keep_new_lines=True)

## BERT WikiNLI

In [13]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikinli-mean-tokens')
# Split the document into sentences
sentences = hazm.sent_tokenize(document)
rtl_print(f"Num sentences: {len(sentences)}")

# Compute the sentence embeddings
embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

# Compute the pair-wise cosine similarities
cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

# Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

# We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


# Print the 3 sentences with the highest scores
rtl_print(f"Article [1000 chars]: {document[:1000]} ...")
rtl_print('- - ' * 50)
rtl_print("Summary:")
rtl_print(' '.join([sentences[idx].strip() for idx in most_central_sentence_indices[:4]]))

## BERT WikiTriplet

In [14]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-wikitriplet-mean-tokens')
# Split the document into sentences
sentences = hazm.sent_tokenize(document)
rtl_print(f"Num sentences: {len(sentences)}")

# Compute the sentence embeddings
embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

# Compute the pair-wise cosine similarities
cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

# Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

# We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


# Print the 3 sentences with the highest scores
rtl_print(f"Article [1000 chars]: {document[:1000]} ...")
rtl_print('- - ' * 50)
rtl_print("Summary:")
rtl_print(' '.join([sentences[idx].strip() for idx in most_central_sentence_indices[:4]]))

## BERT FarsTail

In [15]:
# Load the Sentence-Transformer
embedder = load_st_model('m3hrdadfi/bert-fa-base-uncased-farstail-mean-tokens')
# Split the document into sentences
sentences = hazm.sent_tokenize(document)
rtl_print(f"Num sentences: {len(sentences)}")

# Compute the sentence embeddings
embeddings = embedder.encode(sentences, convert_to_tensor=True, show_progress_bar=True)

# Compute the pair-wise cosine similarities
cos_scores = util.pytorch_cos_sim(embeddings, embeddings).numpy()

# Compute the centrality for each sentence
centrality_scores = degree_centrality_scores(cos_scores, threshold=None)

# We argsort so that the first element is the sentence with the highest score
most_central_sentence_indices = np.argsort(-centrality_scores)


# Print the 3 sentences with the highest scores
rtl_print(f"Article [1000 chars]: {document[:1000]} ...")
rtl_print('- - ' * 50)
rtl_print("Summary:")
rtl_print(' '.join([sentences[idx].strip() for idx in most_central_sentence_indices[:4]]))